## Database Backend - Influx

### Imports

In [1]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
import sys, os
import pandas as pd
from datetime import date, datetime, timedelta
import pytz
import altair as alt


## Set influx connection

In [2]:
# List installed package versions
packList = ["pandas", "altair", "numpy", "influxdb_client"]
for p in packList:
    try:
        mod = __import__(p)
        print(mod.__name__ + " : " + mod.__version__)
    except ImportError:
        print(p + " not installed")

# Get the current working directory
thisDir = os.getcwd()

# List files in the parent directory
parentDir = os.path.abspath(os.path.join(thisDir, os.pardir))
files_in_parent = os.listdir(parentDir)
print("Files in parent directory:", files_in_parent)

pandas : 1.3.4
altair : 4.2.0
numpy : 1.22.3
influxdb_client : 1.30.0
Files in parent directory: ['databases', 'data-presentation', 'data-tools', 'README.md', '.git']


## Read in credentials

Sets a dictionay with:
- 'bucket': "BUCKET_NAME",
- 'org': "ORGANISATION_NAME",
- 'token': "TOKEN",
- 'url': "IP:PORT"


In [3]:
### set up Kenny's credentials (ignore if user!=Kenny)
credDir=os.getcwd()
if os.path.isdir(credDir):
    print("directory found:",credDir)
    sys.path.insert(1, credDir)
    import connectionDetails
    credDict=connectionDetails.GetInfluxCredentials()
    print(credDict)
    print("done.")
else:
    print("no directory found:",credDir)

directory found: /Users/kwraight/CERN_repositories/some-data-science/databases
Getting Influx info.
{'bucket': 'GLADD', 'org': 'PPE', 'token': '6PdYm2oqhOxaA1yRGhpFIypIaNX_etiNNL300Io1NN-OJLD8Apjw1TMQvfq2D3R_KQ5vr-9NB8IVKwuzIrWhlA==', 'url': '194.36.1.20:8086'}
done.


## Query Influx

### get API client

In [4]:
### set client to get access to influx
clientV2_remote = influxdb_client.InfluxDBClient(
   url=credDict['url'],
   token=credDict['token'],
   org=credDict['org']
)

### check buckets

In [5]:
### set bucket api
buckets_api_remote = clientV2_remote.buckets_api()

In [6]:
### Access local or VPN connected ports
### list buckets (by name)
try:
    #print([x.name for x in buckets_api_remote.find_buckets().buckets])
    database_list=[x.name for x in clientV2_remote.buckets_api().find_buckets().buckets]
    print(database_list)
except:
    print("cannot get buckets")

['_monitoring', 'Strips_trashed', 'Strips_stage_pop', 'GLADD', 'REMS', '_monitoring', '_tasks', 'Zigbee', 'Strips_comp_pop', '_tasks', 'coldjig', 'GLADD_TEST', 'pixel_stages', ' ITSDAQ']


### Query data

Example query from influxdb GUI (replace all-caps):
<code>
from(bucket: BUCKET_NAME)
  |> range(start: v.timeRangeStart, stop: v.timeRangeStop)
  |> filter(fn: (r) => r["_measurement"] == DATA_NAME)
  |> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
  |> yield(name: "mean")
</code>

In [7]:
### set query api
query_api_remote = clientV2_remote.query_api()

In [13]:
### build query string
bucketName="REMS"
start_time_str="2023-10-01_13:30:00" # format: '%Y-%m-%d_%H:%M:%S'
# end_time_str="2023-10-02_13:30:00" # format: '%Y-%m-%d_%H:%M:%S'
# set time period, e.g. start time + 1 hr
local = pytz.timezone("Europe/London")
start_time=datetime.strptime(str(start_time_str), '%Y-%m-%d_%H:%M:%S')
start_local = local.localize(start_time, is_dst=None)
start_utc = start_local.astimezone(pytz.utc)
offset_time=3600 #s
end_time=start_time + timedelta(seconds=offset_time)
end_local = local.localize(end_time, is_dst=None)
end_utc = end_local.astimezone(pytz.utc)
# set filters
filters={'_measurement':"data",
         '_field':"humidity",
         'location':"lab_gladd1_env",
         'sensor':"SHT85"
         }
# build string
build_query_str = ' from(bucket: \"'+bucketName+'\") |> range(start: '+start_utc.strftime('%Y-%m-%dT%H:%M:%SZ')+', stop: '+end_utc.strftime('%Y-%m-%dT%H:%M:%SZ')+')'
for k,v in filters.items():
    build_query_str+=' |> filter(fn: (r) => r["'+k+'"] == "'+v+'")'
build_query_str+=' |> yield(name: "mean")'
print(build_query_str)

 from(bucket: "REMS") |> range(start: 2023-10-01T12:30:00Z, stop: 2023-10-01T13:30:00Z) |> filter(fn: (r) => r["_measurement"] == "data") |> filter(fn: (r) => r["_field"] == "humidity") |> filter(fn: (r) => r["location"] == "lab_gladd1_env") |> filter(fn: (r) => r["sensor"] == "SHT85") |> yield(name: "mean")


In [14]:
### get data (into dataframe)
build_result = query_api_remote.query_data_frame(org=credDict['org'], query=build_query_str)
display(build_result)

,result,table,_start,_stop,_time,_value,_field,_measurement,location,sensor,unit_id
0,mean,0,2023-10-01 12:30:00+00:00,2023-10-01 13:30:00+00:00,2023-10-01 12:37:39.656581+00:00,63.49,humidity,data,lab_gladd1_env,SHT85,2
1,mean,0,2023-10-01 12:30:00+00:00,2023-10-01 13:30:00+00:00,2023-10-01 12:38:46.569353+00:00,63.47,humidity,data,lab_gladd1_env,SHT85,2
2,mean,0,2023-10-01 12:30:00+00:00,2023-10-01 13:30:00+00:00,2023-10-01 12:43:14.168789+00:00,63.32,humidity,data,lab_gladd1_env,SHT85,2
3,mean,0,2023-10-01 12:30:00+00:00,2023-10-01 13:30:00+00:00,2023-10-01 12:49:55.526468+00:00,62.96,humidity,data,lab_gladd1_env,SHT85,2
4,mean,0,2023-10-01 12:30:00+00:00,2023-10-01 13:30:00+00:00,2023-10-01 13:01:04.451779+00:00,62.89,humidity,data,lab_gladd1_env,SHT85,2
5,mean,0,2023-10-01 12:30:00+00:00,2023-10-01 13:30:00+00:00,2023-10-01 13:07:45.810726+00:00,63.54,humidity,data,lab_gladd1_env,SHT85,2
6,mean,0,2023-10-01 12:30:00+00:00,2023-10-01 13:30:00+00:00,2023-10-01 13:12:13.376569+00:00,63.79,humidity,data,lab_gladd1_env,SHT85,2
7,mean,0,2023-10-01 12:30:00+00:00,2023-10-01 13:30:00+00:00,2023-10-01 13:23:22.342318+00:00,63.52,humidity,data,lab_gladd1_env,SHT85,2


In [15]:
### copy grafana query
grafana_copy_str='from(bucket: "REMS") \
  |> range(start: 2023-10-01T21:52:26.4Z, stop: 2023-10-02T21:52:26.4Z) \
  |> filter(fn: (r) => r["_measurement"] == "data") \
  |> filter(fn: (r) => r["_field"] == "humidity") \
  |> filter(fn: (r) => r["location"] == "lab_gladd1_env") \
  |> filter(fn: (r) => r["sensor"] == "SHT85") \
  |> filter(fn: (r) => r._value > -900 ) \
  |> aggregateWindow(every: 1m0s, fn: mean, createEmpty: false) \
  |> map(fn: (r) => ({  r with location: "Humidity" })) \
  |> yield(name: "mean")'
print(grafana_copy_str)

from(bucket: "REMS")   |> range(start: 2023-10-01T21:52:26.4Z, stop: 2023-10-02T21:52:26.4Z)   |> filter(fn: (r) => r["_measurement"] == "data")   |> filter(fn: (r) => r["_field"] == "humidity")   |> filter(fn: (r) => r["location"] == "lab_gladd1_env")   |> filter(fn: (r) => r["sensor"] == "SHT85")   |> filter(fn: (r) => r._value > -900 )   |> aggregateWindow(every: 1m0s, fn: mean, createEmpty: false)   |> map(fn: (r) => ({  r with location: "Humidity" }))   |> yield(name: "mean")


In [16]:
### get data (into dataframe)
copy_result = query_api_remote.query_data_frame(org=credDict['org'], query=grafana_copy_str)
display(copy_result)

,result,table,_time,_start,_stop,_field,_measurement,location,sensor,unit_id,_value
0,mean,0,2023-10-01 21:57:00+00:00,2023-10-01 21:52:26.400000+00:00,2023-10-02 21:52:26.400000+00:00,humidity,data,Humidity,SHT85,2,60.42
1,mean,0,2023-10-01 22:08:00+00:00,2023-10-01 21:52:26.400000+00:00,2023-10-02 21:52:26.400000+00:00,humidity,data,Humidity,SHT85,2,60.27
2,mean,0,2023-10-01 22:19:00+00:00,2023-10-01 21:52:26.400000+00:00,2023-10-02 21:52:26.400000+00:00,humidity,data,Humidity,SHT85,2,59.98
3,mean,0,2023-10-01 22:30:00+00:00,2023-10-01 21:52:26.400000+00:00,2023-10-02 21:52:26.400000+00:00,humidity,data,Humidity,SHT85,2,59.92
4,mean,0,2023-10-01 22:41:00+00:00,2023-10-01 21:52:26.400000+00:00,2023-10-02 21:52:26.400000+00:00,humidity,data,Humidity,SHT85,2,59.68
...,...,...,...,...,...,...,...,...,...,...,...
168,mean,0,2023-10-02 21:00:00+00:00,2023-10-01 21:52:26.400000+00:00,2023-10-02 21:52:26.400000+00:00,humidity,data,Humidity,SHT85,2,57.86
169,mean,0,2023-10-02 21:11:00+00:00,2023-10-01 21:52:26.400000+00:00,2023-10-02 21:52:26.400000+00:00,humidity,data,Humidity,SHT85,2,57.88
170,mean,0,2023-10-02 21:22:00+00:00,2023-10-01 21:52:26.400000+00:00,2023-10-02 21:52:26.400000+00:00,humidity,data,Humidity,SHT85,2,57.82
171,mean,0,2023-10-02 21:33:00+00:00,2023-10-01 21:52:26.400000+00:00,2023-10-02 21:52:26.400000+00:00,humidity,data,Humidity,SHT85,2,57.91


## Visualisation
Using altair

In [19]:
### plot data from build query
alt.Chart(build_result).mark_line(point=True).encode(
    x=alt.X('_time'),
    y=alt.Y('_value'),
    tooltip=['_time','_value']
).properties(title="results of custom made query")


alt.Chart(...)

In [20]:
### plot data from grafana query
alt.Chart(copy_result).mark_line(point=True).encode(
    x=alt.X('_time'),
    y=alt.Y('_value'),
    tooltip=['_time','_value']
).properties(title="results of grafana-like query")


alt.Chart(...)